# Summarize Demographics By Plan

In [34]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
# remove col limit
pd.set_option('display.max_columns', None)

DATA_DIR = Path("../data")

In [60]:
def get_demography(df, id_col="ID"):
  df = df[[
    id_col,
    'HISP', 
    'NH_WHITE', 
    'NH_BLACK', 
    'NH_ASIAN', 
    'NH_AMIN', 
    'NH_NHPI', 
    'NH_OTHER',
    'NH_2MORE'
  ]]
  df.loc[:, 'NH_OTHER'] = df[[
    'NH_AMIN', 
    'NH_NHPI', 
    'NH_OTHER',
    'NH_2MORE'
  ]].sum(axis=1)
  df = df.drop(columns=['NH_AMIN', 'NH_NHPI',  'NH_2MORE'])
  return df

precincts12 = gpd.read_file(DATA_DIR / "chicago_precincts_2012" / "chicago_precincts_2012.shp")
precincts12['ID'] = "C" + precincts12['WARD_1'].astype(str).str.zfill(2) + precincts12['PRECINCT_1'].astype(str).str.zfill(3)

ca = gpd.read_file(DATA_DIR / "chicago_community_areas" / "chicago_community_areas.shp").rename(columns={'area_numbe': 'ID'})

wards = gpd.read_file(DATA_DIR / "chicago_wards" / "chicago_wards.shp").rename(columns={'ward': 'ID'})

demography = {
  "ca": get_demography(ca),
  "precinct": get_demography(precincts12),
  "wards": get_demography(wards)
}


In [73]:
from pathlib import Path
import json
PLAN_DIR = Path("../plans_from_figures")
plans_path = PLAN_DIR.glob("*.json")


def get_plan_type(stem):
  if stem.startswith("prec"):
    return "precinct"
  elif stem.startswith("ca"):
    return "ca"
  else:
    return "wards"


def summarize_plan_demographics(plan_object, plan_type):
  demog_df = demography[plan_type]
  # change object to DF
  plan_df = pd.DataFrame({
  "ID": plan_object['assignment'].keys(),
  "DISTRICT": plan_object['assignment'].values()
  })
  return demog_df.merge(plan_df, on="ID", how="left") \
    .groupby("DISTRICT") \
    .agg({
      "HISP": "sum",
      "NH_WHITE": "sum",
      "NH_BLACK": "sum",
      "NH_ASIAN": "sum",
    }).reset_index()


for plan in plans_path:
  with open(plan, "r") as f:
    data = json.load(f)
    plan_type = get_plan_type(plan.stem)
    test_district_demog = summarize_plan_demographics(data, plan_type)
    break

In [74]:
test_district_demog

,DISTRICT,HISP,NH_WHITE,NH_BLACK,NH_ASIAN
0,0,103533.002346,14873.003644,138191.999944,1942.000011
1,1,127797.000111,46544.000014,99075.999978,1584.000007
2,2,39823.810517,27989.000007,202608.647346,5161.000002
3,3,7607.999994,39945.999995,226102.000285,580.999999
4,4,168717.376440,20210.996517,67465.000135,6050.999940
5,5,72244.238208,178942.083298,4405.464228,17716.095035
6,6,50223.000070,132233.000103,44360.999969,37185.000131
7,7,48976.810549,118646.000173,62780.351951,43906.000034
8,8,133880.761957,104224.916789,14032.535734,15669.904883
9,9,31524.999808,192200.999460,18022.000430,16472.999957
